In [2]:
import pandas as pd
import numpy as np
import statistics
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN, OPTICS, Birch
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from numba import njit, prange
import itertools
import math
import haversine as hs
import requests
import json
import time

In [3]:
df = pd.read_excel('CargoSample.xlsx')
df = df.dropna()
dates = df['CargoDate'].unique()

In [4]:
df.head()

,Cargo_ID,DepartmentInfo_ID,CargoDate,SaleInvoice_ID,MainAccount_ID,LatitudePoint,LongitudePoint
0,140218461,18,2023-07-03,1402189460,330623,38.475766,47.063073
1,140218504,18,2023-07-11,14021810299,330623,38.475766,47.063073
2,140218504,18,2023-07-11,14021810493,330623,38.475766,47.063073
3,140218465,18,2023-07-04,1402189463,330627,38.061419,46.322348
4,140218492,18,2023-07-10,14021810212,330627,38.061419,46.322348


In [5]:
df.describe()

,Cargo_ID,DepartmentInfo_ID,SaleInvoice_ID,MainAccount_ID,LatitudePoint,LongitudePoint
count,1.985100e+04,19851.000000,1.985100e+04,1.985100e+04,19851.000000,19851.000000
mean,4.650597e+08,29.075261,7.439222e+09,7.938756e+07,34.532162,51.710861
std,5.518665e+08,15.734324,6.305245e+09,3.038564e+08,3.050551,4.190524
min,1.402411e+07,8.000000,1.402412e+07,3.283910e+05,0.000000,0.000000
25%,1.402143e+08,14.000000,1.402326e+09,3.386380e+05,32.647165,49.580341
50%,1.402332e+08,32.000000,1.402401e+09,3.476130e+05,35.701350,51.395628
75%,1.402081e+09,49.000000,1.402181e+10,3.718650e+05,36.514100,52.712001
max,1.402512e+09,51.000000,1.402513e+10,1.401002e+09,39.652700,62.708922


In [6]:
def compare_machine_to_human_clustering(df):
    number_of_dates = len(dates)
    sum_of_scores = 0
    average = 0
    for date in dates:
        target_date_df = df.loc[df['CargoDate'] == date]
        number_of_visitors = target_date_df.iloc[:, 0].unique().shape[0]

        target_date_df = target_date_df.dropna()
        target_date_df = target_date_df[(target_date_df['LatitudePoint'] > 0) & (target_date_df['LongitudePoint'] > 0)]

        model = AgglomerativeClustering(n_clusters=number_of_visitors, affinity='euclidean', linkage='ward')
        model.fit(target_date_df[['LatitudePoint', 'LongitudePoint']])
        target_date_df['cluster'] = model.labels_

        best_point = 0
        current_point = 0
        for cluster in target_date_df['cluster'].unique():
            target_cluster_df = target_date_df.loc[target_date_df['cluster'] == cluster]
            for cargo_id in target_cluster_df['Cargo_ID'].unique():
                best_point += 1
                target_cargo_df = target_cluster_df.loc[target_cluster_df['Cargo_ID'] == cargo_id]
                current_point += target_cargo_df.shape[0] / target_cluster_df.shape[0]
        sum_of_scores += (current_point/best_point) * 100
    
    average = sum_of_scores / number_of_dates
    return average

compare_machine_to_human_clustering(df)

53.1612729435728

In [7]:
# Example of batch processing
def calculate_distance_batch(points):
    if(len(points) < 2):
        return 0

    requests_string = "http://router.project-osrm.org/route/v1/car/"

    for index, (lat, lon) in enumerate(points):
        requests_string += f"{lon},{lat}"
        if index < len(points) - 1:
            requests_string += ";"

    requests_string += "?overview=false"

    r = requests.get(requests_string)
    routes = json.loads(r.content)

    return routes.get("routes")[0]["distance"]

def build_df_with_cluster(df):
    target_date_df = df.loc[df['CargoDate'] == date]

    number_of_visitors = target_date_df.iloc[:, 0].unique().shape[0]

    target_date_df = target_date_df.dropna()
    target_date_df = target_date_df[target_date_df['LatitudePoint'] > 0]

    model = AgglomerativeClustering(n_clusters=number_of_visitors, affinity='euclidean', linkage='ward')
    model.fit(target_date_df[['LatitudePoint', 'LongitudePoint']])

    target_date_df['cluster'] = model.labels_

    return target_date_df

In [9]:
for date in dates:
    target_date_df = build_df_with_cluster(df)

    #calculate maximum distance between two points in each cluster with latitude and longitude
    try:
        machine_computed_distance = 0
        for cluster in target_date_df['cluster'].unique():
            t_df2 = target_date_df.loc[target_date_df['cluster'] == cluster]
            distance = calculate_distance_batch(t_df2[['LatitudePoint', 'LongitudePoint']].values.tolist())
            machine_computed_distance += distance
                    
        human_computed_distance = 0
        for user_cluster in target_date_df['Cargo_ID'].unique():
            t_df2 = target_date_df.loc[target_date_df['Cargo_ID'] == user_cluster]
            distance = calculate_distance_batch(t_df2[['LatitudePoint', 'LongitudePoint']].values.tolist())
            human_computed_distance += distance
    except: 
        print("Error occured while calculating distance for date", date)
        continue

    print("***************************************************")
    print("AI MAXIMUM DISTANCE RESULT MACHINE", machine_computed_distance, "HUMAN MAXIMUM DISTANCE RESULT", human_computed_distance)
    print("***************************************************")
    #put the result in an excel file
    result_df = pd.DataFrame({'CargoDate': [date], 'Machine': [machine_computed_distance], 'Human': [human_computed_distance]})
    result_df.to_excel('result.xlsx', index=False, header=False)
    time.sleep(50)

***************************************************
AI MAXIMUM DISTANCE RESULT MACHINE 10689820.399999993 HUMAN MAXIMUM DISTANCE RESULT 33742927.79999997
***************************************************
***************************************************
AI MAXIMUM DISTANCE RESULT MACHINE 10573970.799999988 HUMAN MAXIMUM DISTANCE RESULT 38019277.19999997
***************************************************
Error occured while calculating distance for date 2023-07-04T00:00:00.000000000
Error occured while calculating distance for date 2023-07-10T00:00:00.000000000
Error occured while calculating distance for date 2023-07-08T00:00:00.000000000
Error occured while calculating distance for date 2023-07-02T00:00:00.000000000
Error occured while calculating distance for date 2023-07-06T00:00:00.000000000
Error occured while calculating distance for date 2023-07-12T00:00:00.000000000
Error occured while calculating distance for date 2023-07-13T00:00:00.000000000
Error occured while calculat